In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Загрузка данных
data = pd.DataFrame({col: [0] * 1000 for col in [
    'TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD',
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2',
    'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 
    'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12',
    'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10',
    'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7',
    'M8', 'M9'] + [f'V{i}' for i in range(1, 340)]})

# Добавим случайные данные для примера
for col in data.columns:
    if col.startswith(('C', 'D', 'V')):
        data[col] = np.random.randint(0, 10, size=len(data))
    elif col in ['ProductCD', 'card4', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
        data[col] = np.random.choice(['A', 'B', 'C', 'D'], size=len(data))

data['isFraud'] = np.random.randint(0, 2, size=len(data))

# Разделение данных
X = data.drop(['isFraud', 'TransactionID'], axis=1)
y = data['isFraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Преобразование категориальных признаков для XGBoost
cat_features = ['ProductCD', 'card4', 'card6'] + [f'M{i}' for i in range(1, 10)]
for col in cat_features:
    X_train[col] = X_train[col].astype('category').cat.codes
    X_test[col] = X_test[col].astype('category').cat.codes

# 1. CatBoost "из коробки"
cat_features_indices = [X.columns.get_loc(col) for col in cat_features]
train_pool = Pool(X_train, y_train, cat_features=cat_features_indices)
test_pool = Pool(X_test, y_test, cat_features=cat_features_indices)

catboost_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    eval_metric='AUC',
    random_seed=42,
    verbose=100
)

catboost_model.fit(train_pool, eval_set=test_pool)
catboost_pred = catboost_model.predict_proba(test_pool)[:, 1]
catboost_auc = roc_auc_score(y_test, catboost_pred)
print(f"\nCatBoost ROC-AUC (базовый): {catboost_auc:.4f}")

# 2. XGBoost "из коробки"
xgb_model = XGBClassifier(
    objective='binary:logistic',
    random_state=42,
    eval_metric='auc',
    use_label_encoder=False
)

xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict_proba(X_test)[:, 1]
xgb_auc = roc_auc_score(y_test, xgb_pred)
print(f"XGBoost ROC-AUC (базовый): {xgb_auc:.4f}")

# 3. Настроенный XGBoost
tuned_xgb = XGBClassifier(
    objective='binary:logistic',
    random_state=42,
    eval_metric='auc',
    use_label_encoder=False,
    learning_rate=0.01,  # уменьшенный learning rate
    n_estimators=1000,  # больше деревьев
    max_depth=8,        # увеличина глубина
    min_child_weight=5, # регуляризация
    gamma=0.5,         # минимальное снижение потерь для разделения
    subsample=0.8,      # случайные подвыборки строк
    colsample_bytree=0.8, # случайные подвыборки столбцов
    reg_alpha=0.1,      # L1 регуляризация
    reg_lambda=1.0,     # L2 регуляризация
    scale_pos_weight=1  # полезно при дисбалансе классов
)

tuned_xgb.fit(X_train, y_train)
tuned_xgb_pred = tuned_xgb.predict_proba(X_test)[:, 1]
tuned_xgb_auc = roc_auc_score(y_test, tuned_xgb_pred)
print(f"XGBoost ROC-AUC (настроенный): {tuned_xgb_auc:.4f}")

# Сравнение результатов
print("\nСравнение результатов:")
print(f"CatBoost (базовый): {catboost_auc:.4f}")
print(f"XGBoost (базовый): {xgb_auc:.4f}")
print(f"XGBoost (настроенный): {tuned_xgb_auc:.4f}")

if tuned_xgb_auc > catboost_auc:
    print("\nНастроенный XGBoost показал результат лучше CatBoost!")
elif tuned_xgb_auc < catboost_auc:
    print("\nCatBoost показал результат лучше настроенного XGBoost!")
else:
    print("\nРезультаты CatBoost и настроенного XGBoost одинаковы!")

0:	test: 0.5265123	best: 0.5265123 (0)	total: 24.1ms	remaining: 12s
100:	test: 0.5014890	best: 0.5442464 (16)	total: 2.9s	remaining: 11.4s
200:	test: 0.5114005	best: 0.5442464 (16)	total: 5.17s	remaining: 7.7s
300:	test: 0.5010889	best: 0.5442464 (16)	total: 7.46s	remaining: 4.93s
400:	test: 0.5004222	best: 0.5442464 (16)	total: 9.9s	remaining: 2.44s
499:	test: 0.4947331	best: 0.5442464 (16)	total: 12.1s	remaining: 0us

bestTest = 0.544246411
bestIteration = 16

Shrink model to first 17 iterations.

CatBoost ROC-AUC (базовый): 0.5442


C:\Users\den19\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:27:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost ROC-AUC (базовый): 0.4904


C:\Users\den19\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:27:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost ROC-AUC (настроенный): 0.4998

Сравнение результатов:
CatBoost (базовый): 0.5442
XGBoost (базовый): 0.4904
XGBoost (настроенный): 0.4998

CatBoost показал результат лучше настроенного XGBoost!
